In [1]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [2]:
# File paths for the positive and negative training data
train_file_positive = r"C:\Users\cuted\Desktop\deep in\datasets\TrainingDataNegative.txt"
train_file_negative = r"C:\Users\cuted\Desktop\deep in\datasets\TrainingDataNegative.txt"

# Load the positive data
with open(train_file_positive, 'r', encoding='utf-8') as file:
    train_data_positive = file.readlines()

# Load the ne gative data
with open(train_file_negative, 'r', encoding='utf-8') as file:
    train_data_negative = file.readlines()


In [3]:
# Combine positive and negative data
train_data = train_data_positive + train_data_negative
# Create labels: 1 for positive, 0 for negative
labels = [1] * len(train_data_positive) + [0] * len(train_data_negative)

# Create a DataFrame
train_df = pd.DataFrame({'text': train_data, 'label': labels})



In [4]:
#remove punc and lower case
train_df['text'] = train_df['text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x.lower()))


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
import pickle

In [6]:
# Initialize Tokenizer, using top 5000 words and a special token for out-of-vocabulary words
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")

# Fit the tokenizer on the training text data
tokenizer.fit_on_texts(train_df['text'])

In [7]:

# Save the tokenizer to a file
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [8]:
# Convert the text to sequences of integers
X = tokenizer.texts_to_sequences(train_df['text'])
X

[[15,
  1923,
  2620,
  3482,
  217,
  9,
  3483,
  6,
  2,
  1,
  17,
  5,
  1,
  224,
  7,
  140,
  16,
  109,
  1352,
  118,
  225,
  6,
  47,
  63,
  1034,
  1923,
  22,
  554,
  1,
  3,
  1,
  26,
  127,
  3,
  525,
  6,
  1132,
  72,
  3681,
  1,
  862],
 [15,
  29,
  9,
  5,
  1,
  1559,
  217,
  12,
  1407,
  1,
  428,
  2290,
  3,
  1,
  66,
  127,
  3,
  809,
  94,
  601,
  6,
  379,
  19,
  20,
  5,
  292,
  3884,
  7,
  1,
  40,
  57,
  28,
  75,
  47,
  4849,
  73,
  2,
  610,
  7,
  2,
  847,
  48,
  1,
  2,
  847,
  1,
  55,
  6,
  5,
  1,
  13,
  362,
  10,
  1145,
  6,
  2527,
  532,
  5,
  2290,
  48,
  1,
  45,
  1104,
  7,
  2384,
  16,
  62,
  124,
  6,
  472,
  162,
  239,
  53,
  16,
  418,
  11],
 [83,
  348,
  306,
  5,
  218,
  7,
  1,
  3,
  2431,
  1,
  1,
  4161,
  21,
  84,
  247,
  1223,
  14,
  334,
  2290,
  4,
  406,
  22,
  2,
  1041,
  1330,
  3,
  8,
  209,
  12,
  5,
  113,
  1643,
  13,
  25,
  5,
  36,
  1865,
  7,
  1,
  1,
  1,
  3,
  1,
  28,


In [9]:
# Pad sequences to ensure they have the same length
X = pad_sequences(X, padding='post', maxlen=100)
X

array([[  15, 1923, 2620, ...,    0,    0,    0],
       [  15,   29,    9, ...,    0,    0,    0],
       [ 100,   27,   49, ...,   12,   14,  359],
       ...,
       [2155, 2149,    3, ..., 1711,  531,  970],
       [  67, 1038,  239, ...,    0,    0,    0],
       [   6,    1,   88, ..., 2381,   41,  138]], dtype=int32)

In [10]:

# Convert labels to a numpy array
y = np.array(train_df['label'])
y

array([1, 1, 1, ..., 0, 0, 0])

In [11]:


# Step 1: Create the ANN model
model = Sequential()

# Step 2: Add an Embedding layer (if your input is text, otherwise skip this if using numeric features)
model.add(Embedding(input_dim=5000, output_dim=128, input_length=100))

# Step 3: Flatten the embedding output to feed into Dense layers
model.add(Flatten())

# Step 4: Add a Dense layer (Hidden Layer)
model.add(Dense(128, activation='relu'))

# Step 5: Add another Dense layer (Hidden Layer)
model.add(Dense(64, activation='relu'))

# Step 6: Add the Output layer with a sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Step 7: Compile the model with an optimizer, loss function, and metrics
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])





c:\Users\cuted\OneDrive\Desktop\deep in\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [12]:
# Step 8: Display the model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [30]:

history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val), verbose=2)


Epoch 1/50
141/141 - 9s - 65ms/step - accuracy: 1.0000 - loss: 1.4267e-04 - val_accuracy: 0.9253 - val_loss: 0.3442
Epoch 2/50
141/141 - 10s - 71ms/step - accuracy: 1.0000 - loss: 9.4411e-05 - val_accuracy: 0.9253 - val_loss: 0.3540
Epoch 3/50
141/141 - 9s - 65ms/step - accuracy: 1.0000 - loss: 6.7385e-05 - val_accuracy: 0.9253 - val_loss: 0.3631
Epoch 4/50
141/141 - 7s - 52ms/step - accuracy: 1.0000 - loss: 4.9899e-05 - val_accuracy: 0.9249 - val_loss: 0.3687
Epoch 5/50
141/141 - 8s - 55ms/step - accuracy: 1.0000 - loss: 4.0772e-05 - val_accuracy: 0.9267 - val_loss: 0.3768
Epoch 6/50
141/141 - 8s - 55ms/step - accuracy: 1.0000 - loss: 2.9845e-05 - val_accuracy: 0.9276 - val_loss: 0.3837
Epoch 7/50
141/141 - 9s - 67ms/step - accuracy: 1.0000 - loss: 2.5466e-05 - val_accuracy: 0.9276 - val_loss: 0.3878
Epoch 8/50
141/141 - 8s - 57ms/step - accuracy: 1.0000 - loss: 2.0147e-05 - val_accuracy: 0.9276 - val_loss: 0.3928
Epoch 9/50
141/141 - 10s - 69ms/step - accuracy: 1.0000 - loss: 1.7347e

In [14]:
# Evaluate the model on the validation data
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Lo ss: {loss}")
print(f"Validation Accuracy: {accuracy}")


71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7097 - loss: 0.6830
Validation Lo ss: 0.6827255487442017
Validation Accuracy: 0.7146666646003723


In [15]:
# Save the trained model
model.save('sentiment_analysis_model.keras')


In [16]:
import pandas as pd
from keras.models import load_model
import re
from keras.preprocessing.sequence import pad_sequences

In [17]:
#make pred load the model
model = load_model('sentiment_analysis_model.keras')

c:\Users\cuted\OneDrive\Desktop\deep in\venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 16 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [18]:
# Load the saved tokenizer (ensure you saved it as a separate file)
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)


In [19]:
# Load the test CSV file
test_df = pd.read_csv(r"C:\Users\cuted\OneDrive\Desktop\deep in\TestReviews.csv")

# Check the column names to verify the correct column
print(test_df.columns)

Index(['review', 'class'], dtype='object')


In [20]:
 #Assuming the correct column is 'review' (update based on the output of the print statement)
test_df['review'] = test_df['review'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x.lower()))


In [21]:
# Tokenize and pad the test data using the same tokenizer from training
X_test = tokenizer.texts_to_sequences(test_df['review'])
X_test = pad_sequences(X_test, padding='post', maxlen=100)


In [22]:

# Make predictions using the loaded model
predictions = model.predict(X_test)

# Convert predictions to binary labels (1 for positive, 0 for negative)
predicted_labels = [1 if pred > 0.5 else 0 for pred in predictions]

136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [23]:
 #Add predictions to the DataFrame
test_df['predicted_label'] = predicted_labels

# Display the test predictions
print(test_df[['review', 'predicted_label']].head())

                                              review  predicted_label
0  fantastic spot for an even or a quite cocktail...                1
1  love love love the calamari its so good and sp...                1
2  love this place stiff martinis and cocktails c...                1
3  its everything a great cocktail bar should be ...                1
4  i came here before a pirates game so it was ar...                1


In [24]:
# Optionally, save the results to a new CSV
test_df.to_csv('predicted_test_reviews.csv', index=False)

In [25]:
df=pd.read_csv(r"C:\Users\cuted\OneDrive\Desktop\deep in\TestReviews.csv")
df

,review,class
0,Fantastic spot for an even or a quite cocktail...,1
1,"Love, love, love the calamari. It's so good an...",1
2,"Love this place. Stiff martinis and cocktails,...",1
3,It's everything a great cocktail bar should be...,1
4,"I came here before a pirates game, so it was a...",1
...,...,...
4316,My wife and I caught this show at the Golden N...,0
4317,This was the dumbest show we have ever seen. ...,0
4318,My girlfriend and I went to this show. It was ...,0
4319,The restrooms looked like after an bombardment...,0
